In [70]:
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vratnam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Create a mapping of the tokenized words into text and viceversa.

- load_and_encode_data_nltk

nltk uses the nltk tokenizer to split the text into sentenses and words. \
https://www.nltk.org/api/nltk.tokenize.html

- tiktoken
[todo]
- sentence piece
[todo] 

In [63]:
def read_data(filename):
    text = None
    with open(filename, "r", encoding='utf-8') as f:
        text = f.read()
        
    return text

In [71]:
def load_and_encode_data_nltk(filename):
    word_to_lookup = {}
    lookup_to_word = {}
    encoded_data = []

    data = read_data(filename)
    print ('length of dataset = ', len(data))    
    tokenized_data = [word_tokenize(w) for w in [s for s in sent_tokenize(data)]]
    vocabulary = set()

    for s in tokenized_data:
        for w in s:
            vocabulary.add(w)

    vocabulary = sorted(vocabulary)

    for c, i in list(zip(vocab, range(len(vocab)))):
        word_to_lookup[c] = i
        lookup_to_word[i] = c

    for s in tokenized_data:
        for w in s:
            encoded_data.append(word_to_lookup[w])        
        
    return word_to_lookup, lookup_to_word, encoded_data

w2l, l2w, tokenized_data = load_and_encode_data_nltk("data/pg1400.txt")

tensor_tokenized_data = torch.tensor(tokenized_data, dtype=torch.long)

length of dataset =  1013924


In [73]:
print(tensor_tokenized_data.shape, tensor_tokenized_data.dtype)
print(tensor_tokenized_data[:100])

torch.Size([229192]) torch.int64
tensor([13697,  1241,   661,  5082,  8781,   647,   537,  1537,  5120,  7431,
         5967, 12111, 12807,  8781,  2302,  2307,  7092, 12111,  1607,  1453,
         2246,  8407,  8902,  9083,  8781, 12111, 13380,  2509,  8622,  4109,
         2246, 13322,  2188,  8622, 10315, 13172,    10,  1779,  8135,  4077,
         7440,     8,  6277,  7440,  2584,  8867,  9937,  7440, 12617, 12111,
        12069,  8781, 12111,  1241,   661,   904,  7119, 13322, 12169,  5120,
         8867,  8837,  2509, 13444,    10,   770, 13477,  2388,  8660,  7883,
         7092, 12111,  1607,  1453,     8, 13477, 13265,  6661, 12279,  3453,
        12111,  7693,  8781, 12111,  4137, 13181, 13477,  2388,  7883,  2752,
        12817, 12169,  5082,    10,  1554,    56,   647,   537,   162,    56])


Test training dataset split

In [74]:
n = int(0.8*len(tensor_tokenized_data))
train_data = tensor_tokenized_data[:n]
test_data = tensor_tokenized_data[n:]

In [75]:
train_data, test_data

(tensor([13697,  1241,   661,  ...,     8,  2246, 13181]),
 tensor([12111, 11712, 12997,  ...,  8589,  5083,    10]))

--------- EOS